In [1]:
import numpy as np
import scipy.io.wavfile as wav
from IPython.display import Audio, display

# Function to convert a message into a binary string
def message_to_bin(message):
    """Convert the message to a binary string."""
    return ''.join(format(ord(c), '08b') for c in message)

# Function to convert a binary string back to a message
def bin_to_message(binary):
    """Convert a binary string back to the original message."""
    message = []
    for i in range(0, len(binary), 8):
        byte = binary[i:i+8]
        message.append(chr(int(byte, 2)))
    return ''.join(message)

# Embed a secret message using a random pattern and multiple LSBs
def embed_message(audio_file, message, output_file, num_lsb=2, seed=42):
    """Embed a secret message into an audio file using a more sophisticated LSB method."""
    # Read the audio file
    rate, data = wav.read(audio_file)
    
    # Convert the message to binary
    message_bin = message_to_bin(message)
    message_length = len(message_bin)
    
    # Check if the message can fit in the audio data
    max_capacity = len(data) * num_lsb
    if message_length > max_capacity:
        raise ValueError("Message is too long to fit in the audio file with the given LSB capacity.")
    
    # Flatten stereo audio to mono if necessary
    if len(data.shape) == 2:
        data = data.mean(axis=1).astype(np.int16)
    
    # Generate a random pattern for embedding
    np.random.seed(seed)
    indices = np.random.choice(len(data), size=message_length // num_lsb, replace=False)
    indices = sorted(indices)  # Ensure indices are in order for extraction
    
    # Embed the message into the selected samples
    data = data.astype(np.int32)  # Avoid overflow during modification
    for i, idx in enumerate(indices):
        for bit in range(num_lsb):
            bit_pos = i * num_lsb + bit
            if bit_pos >= message_length:
                break
            data[idx] &= ~(1 << bit)  # Clear the LSB
            data[idx] |= (int(message_bin[bit_pos]) << bit)  # Set the LSB to the message bit
    
    # Clip the data and save the new audio file
    data = np.clip(data, -32768, 32767).astype(np.int16)
    wav.write(output_file, rate, data)
    
    print(f"Message embedded successfully into {output_file}.")

# Extract a hidden message using the random pattern and multiple LSBs
def extract_message(audio_file, message_length, num_lsb=2, seed=42):
    """Extract the hidden message from an audio file."""
    # Read the audio file
    rate, data = wav.read(audio_file)
    
    # Flatten stereo audio to mono if necessary
    if len(data.shape) == 2:
        data = data.mean(axis=1).astype(np.int16)
    
    # Generate the same random pattern used for embedding
    np.random.seed(seed)
    indices = np.random.choice(len(data), size=message_length // num_lsb, replace=False)
    indices = sorted(indices)
    
    # Extract the binary message from the selected samples
    binary_message = ''
    for idx in indices:
        for bit in range(num_lsb):
            binary_message += str((data[idx] >> bit) & 1)
    
    # Convert the binary message to text
    message = bin_to_message(binary_message[:message_length])
    return message

# Function to play audio in Jupyter Notebook
def play_audio(audio_file):
    """Play the audio file in the Jupyter notebook using IPython.display.Audio."""
    display(Audio(audio_file, autoplay=True))
    print(f"Playing audio: {audio_file}")

# Example usage

# Secret message to be hidden in the audio file
secret_message = "An eye for an eye makes the whole world blind"

# Embed the secret message into the audio file
embed_message('Ex3_sound5.wav', secret_message, 'Ex3_sound5_with_message.wav', num_lsb=2, seed=42)

# Play the audio with the hidden message
play_audio('Ex3_sound5_with_message.wav')

# Extract the hidden message from the audio file
message_length = len(secret_message) * 8  # Calculate message length in bits
extracted_message = extract_message('Ex3_sound5_with_message.wav', message_length, num_lsb=2, seed=42)
print(f"Extracted Message: {extracted_message}")


Message embedded successfully into Ex3_sound5_with_message.wav.


/var/folders/tk/6yk_3q1j66x8ctk6fc_cby240000gn/T/ipykernel_5258/4108029801.py:23: WavFileWarning: Chunk (non-data) not understood, skipping it.
  rate, data = wav.read(audio_file)


Playing audio: Ex3_sound5_with_message.wav
Extracted Message: An eye for an eye makes the whole world blind
